In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import time
# data generator 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from wandb.keras import WandbCallback
import wandb
import tensorflow as tf
import io

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('/kaggle/input/kaggleinputfashion-product-dataset-processed/fashion-product_12.csv')

In [3]:
data

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link
0,27604,Men,Apparel,Bottomwear,Trousers,Khaki,Summer,2012.0,Casual,Scullers Men Khaki Trouser,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
1,14860,Women,Accessories,Wallets,Wallets,Charcoal,Summer,2011.0,Casual,United Colors of Benetton Women Solid Grey Cha...,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
2,38235,Unisex,Accessories,Eyewear,Sunglasses,Brown,Winter,2016.0,Casual,Polaroid Unisex Sunglasses,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
3,30095,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Ed Hardy Men Black and Red Dial Watch,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
4,19629,Men,Apparel,Topwear,Tshirts,White,Summer,2011.0,Casual,United Colors of Benetton Men Printed White TS...,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
...,...,...,...,...,...,...,...,...,...,...,...,...
10795,48609,Women,Accessories,Jewellery,Earrings,Silver,Summer,2013.0,Casual,Lucera Women Silver Earrings,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
10796,53507,Women,Apparel,Innerwear,Bra,Nude,Winter,2015.0,Casual,Amante Nude-Coloured Full-Coverage Bra BFOM12,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/image/style/pr...
10797,50636,Men,Apparel,Innerwear,Innerwear Vests,Grey Melange,Winter,2016.0,Casual,Chromozome Men Grey Melange Innerwear Vest,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
10798,52647,Women,Accessories,Bags,Handbags,Black,Summer,2012.0,Casual,Mod'acc Women Black Handbag,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...


In [4]:
train_df, test_df = train_test_split(data,\
                                     test_size=0.2,\
                                     random_state=42,\
                                     stratify=data['subCategory'])
valid_df, test_df = train_test_split(test_df,\
                                      test_size=0.5,\
                                      random_state=42,\
                                      stratify=test_df['subCategory'])

In [5]:
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [6]:
train_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link
0,2500,Men,Footwear,Shoes,Casual Shoes,Navy Blue,Fall,2010.0,Casual,Numero Uno Men's Casual Navy Blue Canvas Shoe,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
1,36308,Unisex,Footwear,Flip Flops,Flip Flops,Yellow,Summer,2012.0,Casual,Strapless Unisex Yellow Flip Flops,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
2,54507,Women,Accessories,Bags,Handbags,Pink,Summer,2012.0,Casual,Lino Perros Women Pink Handbag,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
3,26809,Men,Apparel,Bottomwear,Trousers,Charcoal,Summer,2013.0,Formal,Arrow Men Charcoal Flat Front Trousers,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
4,19271,Men,Accessories,Watches,Watches,Steel,Winter,2016.0,Casual,Guess Men Chase Steel Watch,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
...,...,...,...,...,...,...,...,...,...,...,...,...
8635,44045,Men,Personal Care,Fragrance,Perfume and Body Mist,Red,Spring,2017.0,Casual,Hugo Men Energise Perfume,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
8636,43813,Women,Accessories,Jewellery,Earrings,Purple,Summer,2012.0,Casual,Royal Diadem Purple Earrings,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
8637,28275,Men,Personal Care,Fragrance,Deodorant,Grey,Spring,2017.0,Casual,United Colors of Benetton Men Deo,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...
8638,28579,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,ADIDAS Originals Men Black Dial Watch ADH2644,/kaggle/input/fashion-product-images-dataset/f...,http://assets.myntassets.com/v1/images/style/p...


In [7]:
datagen = ImageDataGenerator(rescale=1/255.,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            rotation_range=20,
                            brightness_range=(0.9,1.1))

train_generator = datagen.flow_from_dataframe(dataframe=train_df,
                                             target_size=(224,224),
                                             x_col='filename',
                                            y_col='subCategory',
                                             class_mode='categorical',
                                             batch_size=32,
                                             shuffle=True,
                                              seed=42)


test_datagen = ImageDataGenerator(rescale=1/255.)
valid_generator = test_datagen.flow_from_dataframe(dataframe=valid_df,
                                             target_size=(224,224),
                                             x_col='filename',
                                            y_col='subCategory',
                                             class_mode='categorical',
                                             batch_size=32,
                                             shuffle=True,
                                              seed=42)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='subCategory',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 8640 validated image filenames belonging to 12 classes.
Found 1080 validated image filenames belonging to 12 classes.
Found 1080 validated image filenames belonging to 12 classes.


In [8]:
# tmp

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [10]:
def build_model(name, weights_path=None):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='leaky_relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(1024, activation='tanh')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    predictions = Dense(12, activation='softmax')(x)

    model = Model(name=name, inputs=base_model.input, outputs=predictions)
    if weights_path:
        model.load_weights(weights_path)
    return model

In [11]:
NAME = "vgg19_12class-{}".format(int(time.time()))
model = build_model(NAME)
model.summary()

80134624/80134624 [==============================] - 2s 0us/step
Model: "vgg19_12class-1684851600"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                           

In [12]:
wandb.login()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

StdinNotImplementedError: getpass was called, but this frontend does not support input requests.

In [ ]:
lr = 0.01
epochs = 10
## Initlisazie wandb project
wandb.init(project='CBIR-fashion product dataset',
           name= NAME,
          config={
              "learning_rate": lr,
              "Batch_normalization": True,
              "Batch_size" : 32,
              "Dropout" : "0.4",
              "architecture": "VGG19",
              "dataset": "fashion-product-images-dataset",
              "epochs": epochs,
              "data generator" : True
          }
          )

In [ ]:
wandb_callback = WandbCallback()

filepath = "{}_loss_opti.hdf5".format(NAME)

checkpoint1 = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch"
)


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr),
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,
                    validation_data = train_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=epochs,
                    callbacks=[checkpoint1, wandb_callback])

In [ ]:
model.save(filepath)

In [ ]:
score = model.evaluate_generator(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model(filepath)

In [ ]:
score = best_model.evaluate_generator(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
IMAGESIZE = 224
CHANNELS = 3
def image_preprocess(image_path):
    image_orig = cv2.imread(image_path)
    image_arr = cv2.cvtColor(image_orig, cv2.COLOR_BGR2RGB)
    image_arr = cv2.resize(image_arr, (IMAGESIZE, IMAGESIZE))
    image_arr = image_arr/255.
    image_arr = image_arr.reshape(-1, IMAGESIZE, IMAGESIZE, CHANNELS)
    return image_arr

In [ ]:
test_df.filename[1]

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
anchor_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/50892.jpg'
pos_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/45986.jpg'
neg_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/29863.jpg'
image_arr1 = image_preprocess(anchor_path)
image_arr2 = image_preprocess(pos_path)
image_arr3 = image_preprocess(neg_path)
image_lst = [image_arr1, image_arr2, image_arr3]
for i, image_arr in enumerate(image_lst):
    plt.subplot(1, 3, i+1)
    plt.imshow(image_arr[0])
    plt.axis(False)
plt.show()